In [2]:
import pandas, json
from pandas.io.json import json_normalize
import datetime
import numpy as np
pandas.options.display.max_rows = 150

_file = open('polireddit-export.json', 'r')
data = json.loads(_file.read())
submissions = pandas.DataFrame.from_dict(data['submissions'])

s = submissions.groupby('flair').describe()
s


downs  num_comments       score         ups
flair                                                                
Confirmed     count   32.000000     32.000000   32.000000   32.000000
              mean     5.125000     20.218750   43.500000   48.625000
              std      3.034745     12.853352   28.968281   30.302507
              min      0.000000      1.000000    0.000000    0.000000
              25%      3.000000     10.000000   22.250000   27.000000
              50%      5.000000     19.000000   36.000000   42.000000
              75%      7.250000     30.250000   64.750000   67.750000
              max     12.000000     50.000000  124.000000  132.000000
False         count   70.000000     70.000000   70.000000   70.000000
              mean     7.771429     22.714286   30.271429   38.042857
              std      4.407410     21.631393   21.011330   22.396581
              min      0.000000      1.000000    0.000000    0.000000
              25%      5.000000      8.000000   14.250000   22.250000
              50%      7.000000     15.000000   27.500000   35.000000
              75%     11.000000     28.750000   38.750000   48.750000
              max     21.000000    123.000000   99.000000  106.000000
Half True     count   18.000000     18.000000   18.000000   18.000000
              mean     8.333333     26.666667   25.111111   33.444444
              std      4.445751     19.484534   17.752842   18.579629
              min      3.000000      8.000000    2.000000    7.000000
              25%      4.250000     12.500000   10.500000   17.500000
              50%      7.500000     24.000000   21.000000   32.000000
              75%     11.750000     32.250000   35.000000   44.750000
              max     17.000000     90.000000   68.000000   72.000000
Mod Post/Meta count   44.000000     44.000000   44.000000   44.000000
              mean     5.022727     43.022727   37.568182   42.590909
              std      3.507392    136.308216   49.821728   51.019521
              min      0.000000      0.000000    0.000000    0.000000
              25%      2.750000      3.750000   13.000000   17.500000
              50%      5.000000      9.000000   23.000000   30.000000
              75%      7.000000     17.000000   30.500000   34.000000
              max     20.000000    759.000000  252.000000  260.000000
Mostly False  count   13.000000     13.000000   13.000000   13.000000
              mean     6.076923     18.384615   22.230769   28.307692
              std      4.590961     10.966148    8.516199   10.656646
              min      2.000000      6.000000   11.000000   15.000000
              25%      4.000000      9.000000   15.000000   20.000000
              50%      5.000000     15.000000   22.000000   25.000000
              75%      6.000000     23.000000   26.000000   35.000000
              max     20.000000     40.000000   42.000000   49.000000
Mostly True   count   21.000000     21.000000   21.000000   21.000000
              mean     5.904762     23.238095   37.047619   42.952381
              std      3.845839     23.645094   22.758463   23.620915
              min      1.000000      2.000000    7.000000   10.000000
              25%      3.000000      8.000000   19.000000   26.000000
              50%      6.000000     15.000000   34.000000   40.000000
              75%      7.000000     26.000000   51.000000   57.000000
              max     15.000000     93.000000   94.000000   97.000000
None          count  179.000000    179.000000  179.000000  179.000000
              mean     4.564246     13.268156   18.480447   23.044693
              std      4.535758     29.429513   17.070357   18.414339
              min      0.000000      0.000000    0.000000    0.000000
              25%      1.000000      2.500000    5.000000   12.000000
              50%      3.000000      8.000000   16.000000   22.000000
              75%      6.500000     13.000000   26.500000   31.000000
              max     36.000000    3

In [28]:
_file = open('comment_predict.json', 'r')
data = json.loads(_file.read())
comments = pandas.DataFrame.from_dict(data)
comments = comments[comments['1=Comment; 2=Subcomment'] == 1]
comments = comments[((comments['argument_type'] == '1') | 
                    (comments['argument_type'] == '2')) |
                    (comments['argument_type'] == '3')
                   ]
def action(x):
    return '6' in x
comments['action'] = comments['action'].apply(action)
comments = comments[comments['action'] == True]

def toBinary(x):
    return 1 if (x == '1' or x == '2') else 0
comments['argument_type'] = comments['argument_type'].apply(toBinary)
comments.groupby('argument_type').count()



,1=Comment; 2=Subcomment,Submission,action,author,comment_id,commment_num,created,evidence_source,is_moderator,note,parent_id,score,stance,text
argument_type,,,,,,,,,,,,,,
0,859,859,859,859,859,859,859,855,859,12,859,859,859,859
1,1078,1078,1078,1078,1078,1078,1078,1077,1078,36,1078,1078,1078,1078


In [31]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, r2_score
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import sklearn.decomposition
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold

# Start of the model prediction section

bow_transformer = CountVectorizer(stop_words='english').fit(comments['text'])
data_total_text = bow_transformer.transform(comments['text'])
pca = sklearn.decomposition.TruncatedSVD(n_components=50)
data_total_text = pca.fit_transform(data_total_text)
argument = comments['argument_type']
out_text = ("Reddit dataset has {} data points with {} variables each.".format(*data_total_text.shape))
print out_text

from sklearn.cross_validation import train_test_split as tts
from sklearn.cross_validation import cross_val_score
X_train, X_test, y_train, y_test = tts(data_total_text, argument, test_size=0.25,random_state=1)

def evaluate_cross_validation(clf, X, y, K):
    cv = KFold(len(y), K, shuffle=True, random_state=12)
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    print '-Accuracy'
    print(scores)
    print("Mean score: {0:.3f} (+/-{1:.3f})".format(scores.mean(), scores.std()))
    print '-Recall'
    scores = cross_val_score(clf, X, y, cv=cv, scoring='recall_weighted')
    print(scores)
    print("Mean score: {0:.3f} (+/-{1:.3f})".format(scores.mean(), scores.std()))
    print '-Precision'
    scores = cross_val_score(clf, X, y, cv=cv, scoring='precision_weighted')
    print(scores)
    print("Mean score: {0:.3f} (+/-{1:.3f})".format(scores.mean(), scores.std()))

'''
def train_classifier(clf, X_train, y_train):
    clf.fit(X_train, y_train)

def predict_labels(clf, feature, target):
    y_pred = clf.predict(feature)
    return y_pred, f1_score(target, y_pred)

def train_predict(clf, X_train, X_test, y_train, y_test):
    print ('Training a {}'.format(clf.__class__.__name__))
    train_classifier(clf, X_train, y_train)
    f1_train = predict_labels(clf, X_train, y_train)[1]
    y_pred, f1_test = predict_labels(clf, X_test, y_test)
    _results(y_test, y_pred)
    
    
    return f1_train, f1_test

def _results(y_test, pred):
    print 'Accuracy', accuracy_score(y_test, pred)
    print '\nConfusion Matrix\n', confusion_matrix(y_test, pred)
    print '\n Classification Report\n', classification_report(y_test, pred)
'''

rand_state = 12
clf_A = GaussianNB()
clf_B = svm.SVC(random_state=rand_state)
clf_C = GradientBoostingClassifier(random_state=rand_state)



models = [clf_A, clf_B, clf_C]
results = []
predi = 0
for model in models:
    print('******************************\n')
    print ('Training a {}'.format(model.__class__.__name__))
    #train_values = train_predict(model, X_train, X_test, y_train, y_test)
    #results.append(train_values)
    evaluate_cross_validation(model, data_total_text, argument, 5)
    


Reddit dataset has 1937 data points with 50 variables each.
******************************

Training a GaussianNB
-Accuracy
[ 0.59278351  0.52061856  0.63824289  0.55813953  0.54005168]
Mean score: 0.570 (+/-0.042)
-Recall
[ 0.59278351  0.52061856  0.63824289  0.55813953  0.54005168]
Mean score: 0.570 (+/-0.042)
-Precision
[ 0.6775238   0.65565873  0.7390433   0.73587942  0.64139198]
Mean score: 0.690 (+/-0.041)
******************************

Training a SVC
-Accuracy
[ 0.82474227  0.78608247  0.82428941  0.83204134  0.81912145]
Mean score: 0.817 (+/-0.016)
-Recall
[ 0.82474227  0.78608247  0.82428941  0.83204134  0.81912145]
Mean score: 0.817 (+/-0.016)
-Precision
[ 0.82562979  0.79231029  0.82564827  0.84098424  0.81932593]
Mean score: 0.821 (+/-0.016)
******************************

Training a GradientBoostingClassifier
-Accuracy
[ 0.82989691  0.81443299  0.82945736  0.84237726  0.79328165]
Mean score: 0.822 (+/-0.017)
-Recall
[ 0.82989691  0.81443299  0.82945736  0.84237726  0.7932